<a href="https://colab.research.google.com/github/ernanibn/redes_neurais_artificiais/blob/main/Exercicio_4_Ernani_Brandao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  12.3M      0  0:00:06  0:00:06 --:--:-- 18.0M
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [11]:
import os, pathlib, shutil, random
base_dir = pathlib.Path("aclImdb")
train_dir = base_dir / "train"
val_dir = base_dir / "val"
test_dir = base_dir / "test"
for category in ["pos", "neg"]:
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname, val_dir / category / fname)

In [12]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [13]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) T
Invalid input. Must be one of ('y'

In [14]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [20]:
positive = pipe("This is a great movie!")
positive

[{'label': 'POSITIVE', 'score': 0.9998748302459717}]

In [21]:
negative = pipe("This is a bad movie!")
negative

[{'label': 'NEGATIVE', 'score': 0.9997649788856506}]

In [22]:
negative = pipe("This is a sad movie!")
negative

[{'label': 'NEGATIVE', 'score': 0.9990328550338745}]

In [16]:
from sklearn.metrics import accuracy_score
import numpy as np
import time

In [17]:
ypred = []
test_labels = []
start = time.time()
for batch, (inputs, targets) in enumerate(test_ds):
  for i in range(len(inputs)):
    yp = pipe(inputs[i].numpy().decode('UTF-8')[:512])
    test_labels.append(int(targets[i].numpy()==1))
    ypred.append(int(yp[0]['label']=="LABEL_1"))
  print(batch, time.time()-start, accuracy_score(test_labels, ypred))

0 8.183378219604492 0.375
1 17.87849259376526 0.40625
2 26.752189874649048 0.4583333333333333
3 36.38404083251953 0.5234375
4 46.44340682029724 0.50625
5 54.55518364906311 0.5052083333333334
6 64.48107075691223 0.4955357142857143
7 74.19090461730957 0.50390625
8 82.66606426239014 0.5
9 92.88263440132141 0.490625
10 102.20349979400635 0.4772727272727273
11 110.73157930374146 0.4791666666666667
12 120.90311431884766 0.4735576923076923
13 130.58095121383667 0.4799107142857143
14 139.16270303726196 0.4791666666666667
15 148.76753950119019 0.470703125
16 158.08030724525452 0.4797794117647059
17 166.76495504379272 0.4826388888888889
18 176.62886595726013 0.48355263157894735
19 185.32325649261475 0.48125
20 194.99680137634277 0.48660714285714285
21 204.86114025115967 0.4815340909090909
22 213.8924424648285 0.48097826086956524
23 223.1363446712494 0.4778645833333333
24 233.21540451049805 0.4775
25 242.14409732818604 0.47836538461538464
26 251.4720799922943 0.4791666666666667
27 261.31511449813